In [2]:
import numpy as np
import pandas as pd
import random, copy, os, math

In [3]:
def sample_from_p(floating_edge_indices):
    # if there 2 or more floating edges, pick any 2 putting any one in M1 and the other in M2
    # else pick one in M1 and return an empty M2
    if len(floating_edge_indices) >= 2:
        M1 = floating_edge_indices[:1]
        M2 = floating_edge_indices[1:2]
    else:
        M1 = floating_edge_indices
        M2 = np.array([])
    return M1, M2

# works only for a star graph
def dependent_rounding(edge_weights):
    # see: https://www.cs.umd.edu/~samir/grant/jacm06.pdf
    # edge weights is a vector indicating weights of edges from a given request. -1 means no edge exists.
    
    y = np.array([w for w in edge_weights])
    
    initial_degree = np.sum(y[y != -1])
    
    y[np.isclose(y, 0)] = 0.
    y[np.isclose(y, 1)] = 1.

    assert len(y[y < 0]) == 0 and len(y[y > 1]) == 0, \
        f"edge_weights should be between 0 and 1, found {y[y < 0]} and {y[y > 1]}"
    
    floating_edge_indices = np.where(np.logical_and(np.logical_and(y != -1, y != 0), y != 1))[0]
#     print (floating_edge_indices)
    while len(floating_edge_indices) > 0:
        M1, M2 = sample_from_p(floating_edge_indices)
        assert len(M1) <= 1 and len(M2) <= 1
                
        alpha = np.min([1-y[idx] for idx in M1] + [y[idx] for idx in M2])
        beta = np.min([y[idx] for idx in M1] + [1-y[idx] for idx in M2])
        
        assert alpha > 0 and beta > 0
        
#         print ('alpha:{} , beta: {}'.format(alpha, beta))
        
        p1 = beta/(alpha + beta)
        p2 = 1 - p1
        
#         print ('p1:{}, p2: {}'.format(p1, p2))
        
        delta = np.random.choice([alpha, -beta], size=1, p=[p1, p2])[0]
        
        if len(M2) > 0:
#             print ('Floating edges M1: {}, M2: {}'.format(y[M1], y[M2]))
            for idx_m1, idx_m2 in zip(M1, M2):
                y[idx_m1] += delta
                y[idx_m2] += -delta

                if np.isclose(y[idx_m1], 1):    y[idx_m1] = 1
                elif np.isclose(y[idx_m1], 0):    y[idx_m1] = 0
                if np.isclose(y[idx_m2], 1):    y[idx_m2] = 1
                elif np.isclose(y[idx_m2], 0):    y[idx_m2] = 0
        else:
#             print ('Floating edges M1: {}'.format(y[M1]))
            for idx_m1 in M1:
                y[idx_m1] += delta
                
                if np.isclose(y[idx_m1], 1):    y[idx_m1] = 1
                elif np.isclose(y[idx_m1], 0):    y[idx_m1] = 0
        
        assert len(y[y < 0]) == 0 and len(y[y > 1]) == 0
        
        floating_edge_indices = np.where(np.logical_and(np.logical_and(y != -1, y != 0), y != 1))[0]
    
    final_degree = np.sum(y[y != -1])
    
#     print ()
#     print (final_degree, initial_degree)
#     print ()
#     print (y)
    assert final_degree in [math.ceil(initial_degree), math.floor(initial_degree)]
    
    return y